In [1]:
# Configuración inicial de Spark
import sys
import os

# Configurar rutas para el entorno virtual
sys.path.insert(0, '/opt/python-env/lib/python3.11/site-packages')

# Importar findspark y configurar Spark
import findspark
findspark.init('/opt/spark')

import pyspark
from pyspark.sql import SparkSession

print(f"✅ PySpark versión: {pyspark.__version__}")

# Crear sesión de Spark con configuración optimizada
spark = SparkSession.builder \
    .appName("BigDataExample-ELT") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.executor.memory", "1g") \
    .config("spark.driver.memory", "1g") \
    .getOrCreate()

print("✅ Sesión de Spark creada exitosamente")
print(f"🌐 Spark UI: http://localhost:4040")
print(f"🎯 Master: {spark.sparkContext.master}")
print(f"📊 Spark Version: {spark.version}")

✅ PySpark versión: 3.4.1


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/28 17:17:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✅ Sesión de Spark creada exitosamente
🌐 Spark UI: http://localhost:4040
🎯 Master: spark://spark-master:7077
📊 Spark Version: 3.4.1


In [2]:
# Prueba básica con HDFS
import pandas as pd

# Crear datos de prueba
data = [("Alice", 25), ("Bob", 30), ("Charlie", 35)]
df = spark.createDataFrame(data, ["name", "age"])

# Mostrar datos
df.show()

# Guardar en HDFS
df.write.mode("overwrite").parquet("hdfs://namenode:9000/data/output/test_data")
print("✅ Datos guardados en HDFS")

+-------+---+
|   name|age|
+-------+---+
|  Alice| 25|
|    Bob| 30|
|Charlie| 35|
+-------+---+



[Stage 3:======================================>                   (8 + 4) / 12]

✅ Datos guardados en HDFS


In [3]:
# Leer desde HDFS y procesar
df_read = spark.read.parquet("hdfs://namenode:9000/data/output/test_data")

print("Datos leídos desde HDFS:")
df_read.show()

# Operaciones básicas
avg_age = df_read.agg({"age": "avg"}).collect()[0][0]
print(f"Edad promedio: {avg_age}")

spark.stop()

Datos leídos desde HDFS:
+-------+---+
|   name|age|
+-------+---+
|Charlie| 35|
|  Alice| 25|
|    Bob| 30|
+-------+---+

Edad promedio: 30.0
